## Initialization

In [8]:
#This basic example demostrate the LLM response and ChatModel Response

from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
import openai
import os
from dotenv import load_dotenv, find_dotenv

#app.py

from langchain import PromptTemplate


In [9]:
# Load environment variables from the .env file
load_dotenv(find_dotenv())

# Retrieve Azure OpenAI specific configuration from environment variables
OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
OPENAI_API_TYPE = "Azure"
OPENAI_API_BASE = "https://testopenaisaturday.openai.azure.com/"
OPENAI_API_VERSION = "2023-10-01-preview"

print(OPENAI_API_KEY, OPENAI_API_TYPE,OPENAI_API_BASE, OPENAI_API_VERSION )

370e4756680d40a9978934a4f8af3ed9 Azure https://testopenaisaturday.openai.azure.com/ 2023-10-01-preview


In [10]:
# Set the OpenAI library configuration using the retrieved environment variables
openai.api_type = "Azure"
openai.api_base = "https://testopenaisaturday.openai.azure.com/"
openai.api_version = "2023-10-01-preview"
openai.api_key = OPENAI_API_KEY


llm = AzureChatOpenAI( 
    openai_api_key=OPENAI_API_KEY,
    azure_endpoint=OPENAI_API_BASE,
    openai_api_version=OPENAI_API_VERSION,
    deployment_name="assistantPreviewSaturday",
)


In [11]:
#app.py

from langchain import PromptTemplate
import openai
from dotenv import load_dotenv, find_dotenv
import os
from langchain.chat_models import AzureChatOpenAI 
from dotenv import find_dotenv, load_dotenv



In [12]:


llm_temp = AzureChatOpenAI( 
    openai_api_key=OPENAI_API_KEY,
    azure_endpoint=OPENAI_API_BASE,
    openai_api_version=OPENAI_API_VERSION,
    deployment_name="assistantPreviewSaturday",
    temperature=0.7
)



In [13]:
from test_unstructured.unit_utils import assert_round_trips_through_JSON, example_doc_path
from unstructured.chunking.title import chunk_by_title
from unstructured.documents.elements import ElementType

#!pip install unstructured_inference
#!pip install -U langchain-unstructured

## Chunking

In [14]:
import PyPDF2

# Open the PDF file
with open('s3.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    # Iterate through all the pages and extract text
    text = ''
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()

# Print the extracted text
#print(text)


In [31]:
import PyPDF2
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document  # Import the Document class

# Step 1: Extract text from the PDF using PyPDF2
with open('s3.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    # Iterate through all the pages and extract text
    text = ''
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()

# Step 2: Create a Document object
document = Document(page_content=text)

# Step 3: Use CharacterTextSplitter to split the extracted text into chunks
text_splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=0)
texts = text_splitter.split_documents([document])  # Pass in a list of Document objects

# Step 4: Output the split text chunks
for chunk in texts:
    print(chunk.page_content)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [56]:
import PyPDF2
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document  # Import the Document class

# Step 1: Extract text from the PDF using PyPDF2
with open('s3.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    # Iterate through all the pages and extract text
    text = ''
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        extracted_text = page.extract_text()
        if extracted_text:  # Ensure the text extraction is successful
            text += extracted_text

# Step 2: Create a Document object
document = Document(page_content=text)

from langchain.text_splitter import RecursiveCharacterTextSplitter

# Step 3: Use RecursiveCharacterTextSplitter for better splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=200,
    separators=["\n\n", "\n", " "]
)

# Split the document text into smaller chunks
texts = text_splitter.split_documents([document])  # Pass in a list of Document objects

# Step 4: Print the number of chunks
print(f"Number of chunks: {len(texts)}")

# Step 5: Optionally, output the first few chunks to inspect the splitting
for i, chunk in enumerate(texts[:3]):  # Limit to first 3 chunks for display
    print(f"Chunk {i+1}:")
    print(chunk.page_content[:500])  # Print first 500 characters of the chunk
    print("\n" + "-"*80 + "\n")



Number of chunks: 2405
Chunk 1:
User Guide
Amazon Simple Storage Service
API Version 2006-03-01
Copyright © 2024 Amazon Web Services, Inc. and/or its aﬃliates. All rights reserved.Amazon Simple Storage Service User Guide
Amazon Simple Storage Service: User Guide
Copyright © 2024 Amazon Web Services, Inc. and/or its aﬃliates. All rights reserved.
Amazon's trademarks and trade dress may not be used in connection with any product or service 
that is not Amazon's, in any manner that is likely to cause confusion among customers, or

--------------------------------------------------------------------------------

Chunk 2:
Analytics and insights............................................................................................................................5
Strong consistency..................................................................................................................................5
How Amazon S3 works..........................................................

In [73]:
# Semantic Chunker
import PyPDF2
from langchain.schema import Document  # Import the Document class
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# Step 1: Extract text from the PDF using PyPDF2
with open('s3.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    
    # Iterate through all the pages and extract text
    text = ''
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        extracted_text = page.extract_text()
        if extracted_text:  # Ensure the text extraction is successful
            text += extracted_text

# Step 2: Create a Document object
document = Document(page_content=text)

# Initialize the OpenAI embeddings model (required for SemanticChunker)
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key="YOUR_API_KEY"  # Replace with your OpenAI API key
)

# Step 3: Use SemanticChunker for semantic splitting
text_splitter = SemanticChunker(embeddings_model, breakpoint_threshold_type="percentile")

# Split the document text into semantically meaningful chunks
texts = text_splitter.create_documents([document.page_content])

# Step 4: Print the number of chunks
print(f"Number of semantic chunks: {len(texts)}")

# Step 5: Optionally, output the first few chunks to inspect the splitting
for i, chunk in enumerate(texts[:3]):  # Limit to first 3 chunks for display
    print(f"Chunk {i+1}:")
    print(chunk.page_content[:500])  # Print first 500 characters of each chunk
    print("\n" + "-"*80 + "\n")



In [72]:
# Step 4: Output only the first 2-3 split text chunks with truncation to avoid large data output
for chunk in texts[125:127]:  # Display the first 3 chunks
    print(chunk.page_content[:500])  # Print only the first 500 characters of each chunk
    print("\n" + "-"*80 + "\n")


delete directories, and it does not support symbolic links or ﬁle locking. Mountpoint is ideal 
for applications that do not need all of the features of a shared ﬁle system and POSIX-style 
permissions but require Amazon S3's elastic throughput to read and write large S3 datasets. For 
details, see Mountpoint ﬁle system behavior on GitHub. For workloads that require full POSIX 
support, we recommend Amazon FSx for Lustre and its support for linking S3 buckets.
Mountpoint for Amazon S3 is availab

--------------------------------------------------------------------------------

wget download-link
3. (Optional) Verify the authenticity and integrity of the downloaded ﬁle. First, copy the 
appropriate signature URL for your architecture.
x86_64 :
https://s3.amazonaws.com/mountpoint-s3-release/latest/x86_64/mount-s3.rpm.asc
Installing Mountpoint API Version 2006-03-01 85Amazon Simple Storage Service User Guide
ARM64 (Graviton) :
https://s3.amazonaws.com/mountpoint-s3-release/latest/arm64/mo

## Metadata Gen : Topic Tags

In [59]:
# Modify the function to work with strings (plain text chunks) and generate 2-3 tags
def generate_metadata_for_chunks(llm, chunks, max_tokens=2048):
    results = []
    for i, chunk in enumerate(chunks):
        # Truncate the chunk if it exceeds the max_tokens limit
        truncated_chunk = chunk[:max_tokens]  # Now 'chunk' is a string, so we handle it directly
        
        # Create a prompt that requests the LLM to generate only the most relevant 2-3 tags and headings
        prompt = f"""
        The following text is a chunk from a larger document. Please generate the 2-3 most relevant topic headings and tags for this text:
        Text: "{truncated_chunk}"
        """
        
        # Call the LLM with the prompt using the invoke method
        response = llm.invoke([{
            "role": "system",
            "content": prompt
        }])
        
        # Store the result (response with tags and headings)
        results.append({
            "chunk_index": i,
            "chunk_text": truncated_chunk,
            "metadata": response
        })
    
    return results

# Display the results with truncated chunk text and the most relevant metadata
def display_metadata_results(results):
    for result in results:
        print(f"Chunk Index: {result['chunk_index']}")
        print("Chunk Text (truncated):")
        print(result['chunk_text'][:1000])  # Display only the first 1000 characters of the chunk text
        print("\nGenerated Metadata:")
        
        # Extract and display the topic headings and tags from the metadata
        metadata_content = result['metadata'].content
        
        # Format the metadata output
        print(f"{metadata_content}")
        print("\n" + "-"*80 + "\n")


In [60]:
# Function to pick a specific chunk by index
def pick_chunks_by_index(indices, texts):
    selected_chunks = [texts[i].page_content for i in indices if i < len(texts)]
    return selected_chunks

# Example usage: pick chunk(s) by index
indices = [1004]  # You can manually update this list with the indices of the chunks you want to select

# Get the selected chunks based on the provided indices
selected_chunks = pick_chunks_by_index(indices, texts)

# Generate metadata for the selected chunks
results = generate_metadata_for_chunks(llm, selected_chunks)

# Output the results to see how it works
display_metadata_results(results)


Chunk Index: 0
Chunk Text (truncated):
"s3:GetObject" 
         ], 
         "Resource": [ 
            "arn:aws:s3::: amzn-s3-demo-bucket1 /*" 
         ] 
      }   
   ]
}
Step 2: Do the Account B tasks
Now that Account B has permissions to perform operations on Account A's bucket, the Account B 
administrator does the following:
•Uploads an object to Account A's bucket
•Adds a grant in the object ACL to allow Account A, the bucket owner, full control
Walkthroughs using policies API Version 2006-03-01 1010Amazon Simple Storage Service User Guide
Using the AWS CLI
1. Using the put-object  AWS CLI command, upload an object. The --body  parameter in the 
command identiﬁes the source ﬁle to upload. For example, if the ﬁle is on the C: drive of a 
Windows machine, specify c:\HappyFace.jpg . The --key parameter provides the key name 
for the object.
aws s3api put-object --bucket amzn-s3-demo-bucket1  --key HappyFace.jpg --body 
 HappyFace.jpg --profile AccountBadmin
2. Add a grant to the 

## Embedings : Split and add metadata with chunks to Vector DB

In [61]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

# Assuming `generate_metadata_for_chunks` gives us topics and tags for each chunk
def create_documents_with_metadata(chunks, metadata):
    documents = []
    for i, chunk in enumerate(chunks):
        doc_metadata = {
            "topics": metadata[i].get("topics", []),  # Assuming metadata contains 'topics'
            "tags": metadata[i].get("tags", [])  # Assuming metadata contains 'tags'
        }
        document = Document(page_content=chunk.page_content, metadata=doc_metadata)
        documents.append(document)
    return documents




In [62]:
# Load the PDF document (replace with your own PDF file)
directory = os.getcwd()
file_name = directory + "/s3.pdf"  # Example file name
loader = PyPDFLoader(file_name)
pages = loader.load_and_split()

# Generate metadata for each chunk (topics and tags)
metadata = generate_metadata_for_chunks(llm, [page.page_content for page in pages])

# Create documents with metadata
documents = create_documents_with_metadata(pages, metadata)


In [65]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings


# Initialize the embeddings model using Azure OpenAI
embeddings = AzureOpenAIEmbeddings(
    deployment = "AzureAdaLangchain",
    model = "text-embedding-ada-002",
    api_key= "370e4756680d40a9978934a4f8af3ed9",
    openai_api_version="2023-10-01-preview",
    azure_endpoint="https://testopenaisaturday.openai.azure.com/",
    openai_api_type="azure",
    chunk_size=1
)

# Use FAISS to index the document chunks along with their metadata
db = FAISS.from_documents(documents=documents, embedding=embeddings)

# Save the FAISS index locally
db.save_local(directory + "/faiss_index_s3")

## Integrate LLM with embedded DB

In [67]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain

# Initialize the LLM
llm = AzureChatOpenAI( 
    openai_api_key=OPENAI_API_KEY,
    azure_endpoint=OPENAI_API_BASE,
    openai_api_version=OPENAI_API_VERSION,
    deployment_name="varelabsAssistant",
    temperature=0.7
)

# Load the FAISS vector store
vector_store = FAISS.load_local(directory + "/faiss_index_s3", embeddings, allow_dangerous_deserialization=True)

# Set up a retriever using similarity search
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [68]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Create the prompt for condensing the question based on chat history
CONDENSE_PROMPT = PromptTemplate.from_template("""
You are an assistant helping to condense a follow-up question in a conversation about AWS S3 installation and troubleshooting. Use the chat history to rephrase the user's follow-up question into a standalone question that includes references to any specific issues or configurations mentioned.

Chat History:
{chat_history}
Follow-Up Input: {question}
Standalone Question:
""")

# Create the prompt for generating the answer based on the documentation
QA_PROMPT = PromptTemplate.from_template("""
You are a helpful assistant providing guidance on AWS S3 installation and troubleshooting. Use the provided documentation to give accurate, concise answers based on the user's questions. Use the conversation history to understand the user's current context and refer to any mentioned topics or configurations.

Documentation Context:
{context}

Conversation History:
{chat_history}

User's Question: {question}
Assistant's Response:
""")

# Initialize the Conversational Retrieval Chain with the LLM and retriever
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    condense_question_prompt=CONDENSE_PROMPT,
    combine_docs_chain_kwargs={'prompt': QA_PROMPT},
    return_source_documents=True,
    verbose=False
)

# Function to ask a question and maintain conversation context
def ask_question_with_context(qa, question, chat_history):
    result = qa({"question": question, "chat_history": chat_history})
    print("Assistant's Response:", result["answer"])
    chat_history.append((question, result["answer"]))
    return chat_history


In [69]:
# Initialize an empty chat history
chat_history = []


In [70]:

# Example questions relevant to AWS S3 documentation, installation, and troubleshooting
question = "How do I set up an S3 bucket on AWS?"
chat_history = ask_question_with_context(qa, question, chat_history)

question = "I'm getting a permission error while accessing my S3 bucket. How can I fix this?"
chat_history = ask_question_with_context(qa, question, chat_history)

question = "How do I configure access policies for S3?"
chat_history = ask_question_with_context(qa, question, chat_history)

question = "How do I enable versioning on my S3 bucket?"
chat_history = ask_question_with_context(qa, question, chat_history)

C:\Users\prana\AppData\Local\Temp\ipykernel_16948\2120289748.py:40: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa({"question": question, "chat_history": chat_history})


Assistant's Response: To set up an S3 bucket on AWS, follow these steps:

1. **Sign in to the AWS Management Console**:
   - Go to the Amazon S3 console at [https://console.aws.amazon.com/s3/](https://console.aws.amazon.com/s3/).

2. **Choose the AWS Region**:
   - In the navigation bar at the top of the page, select the name of the currently displayed AWS Region.
   - Choose the Region where you want to create the bucket. Selecting a Region close to you can help minimize latency and costs and address regulatory requirements.

3. **Navigate to Buckets**:
   - In the left navigation pane, click on **Buckets**.

4. **Create a Bucket**:
   - Click on **Create bucket**.
   - The Create bucket page will open.

5. **Configure Bucket Settings**:
   - **Bucket name**: Enter a name for your bucket (e.g., `tutorial-bucket`). Ensure that the name is unique and adheres to the [Bucket naming rules](https://docs.aws.amazon.com/AmazonS3/latest/userguide/bucketnamingrules.html).
   - **Region**: Selec